In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import pickle
sys.path.append('../../')
print(sys.path)
import IPython.display as ipd
import numpy as np
from tools.libaudio.feature import melspectrogram
from tools.libaudio.display import show_spec, show_mel
from mlutils.utils import plot, plots, to_onehot
from models.mlmodeldic import best_model_path_and_settings
import torch
from tts.Alternative.neuravoice2 import NewTTS, Encoder
from tts.Alternative.train import train_char2wav
from tts.Alternative.dataset import NeuraVoiceDataset
from torch.utils.data import DataLoader
import time

['/var/Sources/experiments/core/tts/Alternative', '/home/kaz/anaconda3/envs/core/lib/python37.zip', '/home/kaz/anaconda3/envs/core/lib/python3.7', '/home/kaz/anaconda3/envs/core/lib/python3.7/lib-dynload', '', '/home/kaz/.local/lib/python3.7/site-packages', '/home/kaz/anaconda3/envs/core/lib/python3.7/site-packages', '/home/kaz/anaconda3/envs/core/lib/python3.7/site-packages/Mako-1.0.7-py3.7.egg', '/home/kaz/anaconda3/envs/core/lib/python3.7/site-packages/IPython/extensions', '/home/kaz/.ipython', '../../']


In [ ]:
cuda = 2
batch_size = 16
model_version = 1
model_name = f'newtts_ver{model_version}'

In [ ]:
dataset = NeuraVoiceDataset(batch_size=batch_size)
loader = DataLoader(
    dataset, batch_size=batch_size, shuffle=False, collate_fn=dataset.char_to_wav, drop_last=False, num_workers=1)

In [ ]:
model_path, settings = best_model_path_and_settings(model_name, 'loss_ave', is_lower_better=True)
model_path, settings

In [ ]:
device = f'cuda:{cuda}'

In [ ]:
use_best_model = False

In [ ]:
if model_path and use_best_model:
    encoder = Encoder.init_from_settings(settings, model_path, device=device, **{'version': model_version})
    decoder = NewTTS.init_from_settings(settings, model_path, device=device, **{'version': model_version})
else:                             
    encoder = Encoder(K=10, hidden_size=512, bit=9, version=model_version, device=device)
    decoder = NewTTS(K=10, hidden_size=512, bit=9, version=model_version, device=device)

In [ ]:
losses, loss_aves, model = train_char2wav(
    encoder, decoder, loader, n_epoch=100, model_name=model_name,
    seqlen=1000, stride=100, device=device, lr=1e-3, verbose=False)

In [ ]:
#plot(loss_aves)

In [ ]:
#plots([losses, loss_aves], labels=['losses', 'loss_aves'])